In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset
import torch
import numpy as np

c:\Users\SC\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
checkpoint = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'

model = AutoModelForCausalLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [2]:
from transformers import pipeline, AutoModel

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
print('using', device)

using cpu


In [5]:
context = 'A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. options: {A: Ampicillin, B: Ceftriaxone, C: Ciprofloxacin, D: Doxycycline, E: Nitrofurantoin}'
question = 'Which of the following is the best treatment for this patient?'

prompt = 'A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient? options: {A: Ampicillin, B: Ceftriaxone, C: Ciprofloxacin, D: Doxycycline, E: Nitrofurantoin}'

In [6]:
# mod_pipeline = pipeline(
#     'text-generation', model=model, tokenizer=tokenizer
# )

med_pipeline = pipeline(
    'text-generation', model='medalpaca/medalpaca-7b', tokenizer='medalpaca/medalpaca-7b', max_length=500
)

Loading checkpoint shards: 100%|██████████| 3/3 [01:11<00:00, 23.74s/it]
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
sample_prompt = """Answer the question using the context below.
Context: Gazpacho is a cold soup and drink made of raw, blended vegetables. Most gazpacho includes stale bread, tomato, cucumbers, onion, bell peppers, garlic, olive oil, wine vinegar, water, and salt. Northern recipes often include cumin and/or pimentón (smoked sweet paprika). Traditionally, gazpacho was made by pounding the vegetables in a mortar with a pestle; this more laborious method is still sometimes used as it helps keep the gazpacho cool and avoids the foam and silky consistency of smoothie versions made in blenders or food processors.
Question: What modern tool is used to make gazpacho?
Answer:
"""

sample_output = mod_pipeline(sample_prompt)

NameError: name 'mod_pipeline' is not defined

In [7]:
med_prompt = """Answer the question using the context below.
Context: A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus.
Question: Which of the following is the best treatment for this patient? options: {A: Ampicillin, B: Ceftriaxone, C: Ciprofloxacin, D: Doxycycline, E: Nitrofurantoin}
Answer:
"""
med_output = med_pipeline(med_prompt)
med_output


[{'generated_text': 'Answer the question using the context below.\nContext: A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus.\nQuestion: Which of the following is the best treatment for this patient? options: {A: Ampicillin, B: Ceftriaxone, C: Ciprofloxacin, D: Doxycycline, E: Nitrofurantoin}\nAnswer:\nA'}]

In [9]:
response1 = """Answer the question using the context below.
Context: A 3-month-old baby died suddenly at night while asleep. His mother noticed that he had died only after she awoke in the morning. No cause of death was determined based on the autopsy.
Question: Which of the following precautions could have prevented the death of the baby? Options: {A: Placing the infant in a supine position on a firm mattress while sleeping, B: Routine postnatal electrocardiogram (ECG), C: Keeping the infant covered and maintaining a high room temperature, D: Application of a device to maintain the sleeping position, E: Avoiding pacifier use during sleep}
Answer:
"""

med_output2 = med_pipeline(response1)
med_output2

[{'generated_text': 'Answer the question using the context below.\nContext: A 3-month-old baby died suddenly at night while asleep. His mother noticed that he had died only after she awoke in the morning. No cause of death was determined based on the autopsy.\nQuestion: Which of the following precautions could have prevented the death of the baby? Options: {A: Placing the infant in a supine position on a firm mattress while sleeping, B: Routine postnatal electrocardiogram (ECG), C: Keeping the infant covered and maintaining a high room temperature, D: Application of a device to maintain the sleeping position, E: Avoiding pacifier use during sleep}\nAnswer:\nA'}]

In [10]:
response2 = """Answer the question using the context below.
Context: A mother brings her 3-week-old infant to the pediatrician's office because she is concerned about his feeding habits. He was born without complications and has not had any medical problems up until this time. However, for the past 4 days, he has been fussy, is regurgitating all of his feeds, and his vomit is yellow in color. On physical exam, the child's abdomen is minimally distended but no other abnormalities are appreciated.
Question: Which of the following embryologic errors could account for this presentation? Options: {A: Abnormal migration of ventral pancreatic bud, B: Complete failure of proximal duodenum to recanalize, C: Error in neural crest cell migration, D: Abnormal hypertrophy of the pylorus, E: Failure of lateral body folds to move ventrally and fuse in the midline}
Answer:
"""
med_output3 = med_pipeline(response2)
med_output3

[{'generated_text': "Answer the question using the context below.\nContext: A mother brings her 3-week-old infant to the pediatrician's office because she is concerned about his feeding habits. He was born without complications and has not had any medical problems up until this time. However, for the past 4 days, he has been fussy, is regurgitating all of his feeds, and his vomit is yellow in color. On physical exam, the child's abdomen is minimally distended but no other abnormalities are appreciated.\nQuestion: Which of the following embryologic errors could account for this presentation? Options: {A: Abnormal migration of ventral pancreatic bud, B: Complete failure of proximal duodenum to recanalize, C: Error in neural crest cell migration, D: Abnormal hypertrophy of the pylorus, E: Failure of lateral body folds to move ventrally and fuse in the midline}\nAnswer:\nC"}]

In [11]:
response3 = """Answer the question using the context below.
Context: A pulmonary autopsy specimen from a 58-year-old woman who died of acute hypoxic respiratory failure was examined. She had recently undergone surgery for a fractured femur 3 months ago. Initial hospital course was uncomplicated, and she was discharged to a rehab facility in good health. Shortly after discharge home from rehab, she developed sudden shortness of breath and had cardiac arrest. Resuscitation was unsuccessful. On histological examination of lung tissue, fibrous connective tissue around the lumen of the pulmonary artery is observed.
Question: Which of the following is the most likely pathogenesis for the present findings? Options: {A: Thromboembolism, B: Pulmonary ischemia, C: Pulmonary hypertension, D: Pulmonary passive congestion, E: Pulmonary hemorrhage}
Answer:
"""
med_output4 = med_pipeline(response3)
med_output4

[{'generated_text': 'Answer the question using the context below.\nContext: A pulmonary autopsy specimen from a 58-year-old woman who died of acute hypoxic respiratory failure was examined. She had recently undergone surgery for a fractured femur 3 months ago. Initial hospital course was uncomplicated, and she was discharged to a rehab facility in good health. Shortly after discharge home from rehab, she developed sudden shortness of breath and had cardiac arrest. Resuscitation was unsuccessful. On histological examination of lung tissue, fibrous connective tissue around the lumen of the pulmonary artery is observed.\nQuestion: Which of the following is the most likely pathogenesis for the present findings? Options: {A: Thromboembolism, B: Pulmonary ischemia, C: Pulmonary hypertension, D: Pulmonary passive congestion, E: Pulmonary hemorrhage}\nAnswer:\nA'}]

In [12]:
response4 = """Answer the question using the context below.
Context: A 20-year-old woman presents with menorrhagia for the past several years. She says that her menses “have always been heavy”, and she has experienced easy bruising for as long as she can remember. Family history is significant for her mother, who had similar problems with bruising easily. The patient's vital signs include: heart rate 98/min, respiratory rate 14/min, temperature 36.1°C (96.9°F), and blood pressure 110/87 mm Hg. Physical examination is unremarkable. Laboratory tests show the following: platelet count 200,000/mm3, PT 12 seconds, and PTT 43 seconds.
Question: Which of the following is the most likely cause of this patient’s symptoms? Options: {A: Factor V Leiden, B: Hemophilia A, C: Lupus anticoagulant, D: Protein C deficiency, E: Von Willebrand disease}
Answer:
"""
med_output5 = med_pipeline(response4)
med_output5

[{'generated_text': "Answer the question using the context below.\nContext: A 20-year-old woman presents with menorrhagia for the past several years. She says that her menses “have always been heavy”, and she has experienced easy bruising for as long as she can remember. Family history is significant for her mother, who had similar problems with bruising easily. The patient's vital signs include: heart rate 98/min, respiratory rate 14/min, temperature 36.1°C (96.9°F), and blood pressure 110/87 mm Hg. Physical examination is unremarkable. Laboratory tests show the following: platelet count 200,000/mm3, PT 12 seconds, and PTT 43 seconds.\nQuestion: Which of the following is the most likely cause of this patient’s symptoms? Options: {A: Factor V Leiden, B: Hemophilia A, C: Lupus anticoagulant, D: Protein C deficiency, E: Von Willebrand disease}\nAnswer:\nE"}]

In [13]:
response5 = """Answer the question using the context below.
Context: A 40-year-old zookeeper presents to the emergency department complaining of severe abdominal pain that radiates to her back, and nausea. The pain started 2 days ago and slowly increased until she could not tolerate it any longer. Past medical history is significant for hypertension and hypothyroidism. Additionally, she reports that she was recently stung by one of the zoo’s smaller scorpions, but did not seek medical treatment. She takes aspirin, levothyroxine, oral contraceptive pills, and a multivitamin daily. Family history is noncontributory. Today, her blood pressure is 108/58 mm Hg, heart rate is 99/min, respiratory rate is 21/min, and temperature is 37.0°C (98.6°F). On physical exam, she is a well-developed, obese female that looks unwell. Her heart has a regular rate and rhythm. Radial pulses are weak but symmetric. Her lungs are clear to auscultation bilaterally. Her lateral left ankle is swollen, erythematous, and painful to palpate. An abdominal CT is consistent with acute pancreatitis.
Question: Which of the following is the most likely etiology for this patient’s disease? Options: {A: Aspirin, B: Oral contraceptive pills, C: Scorpion sting, D: Hypothyroidism, E: Obesity}
Answer:
"""
med_output6 = med_pipeline(response5)
med_output6

[{'generated_text': 'Answer the question using the context below.\nContext: A 40-year-old zookeeper presents to the emergency department complaining of severe abdominal pain that radiates to her back, and nausea. The pain started 2 days ago and slowly increased until she could not tolerate it any longer. Past medical history is significant for hypertension and hypothyroidism. Additionally, she reports that she was recently stung by one of the zoo’s smaller scorpions, but did not seek medical treatment. She takes aspirin, levothyroxine, oral contraceptive pills, and a multivitamin daily. Family history is noncontributory. Today, her blood pressure is 108/58 mm Hg, heart rate is 99/min, respiratory rate is 21/min, and temperature is 37.0°C (98.6°F). On physical exam, she is a well-developed, obese female that looks unwell. Her heart has a regular rate and rhythm. Radial pulses are weak but symmetric. Her lungs are clear to auscultation bilaterally. Her lateral left ankle is swollen, eryt

In [14]:
response6 = """Answer the question using the context below.
Context: A 25-year-old primigravida presents to her physician for a routine prenatal visit. She is at 34 weeks gestation, as confirmed by an ultrasound examination. She has no complaints, but notes that the new shoes she bought 2 weeks ago do not fit anymore. The course of her pregnancy has been uneventful and she has been compliant with the recommended prenatal care. Her medical history is unremarkable. She has a 15-pound weight gain since the last visit 3 weeks ago. Her vital signs are as follows: blood pressure, 148/90 mm Hg; heart rate, 88/min; respiratory rate, 16/min; and temperature, 36.6℃ (97.9℉). The blood pressure on repeat assessment 4 hours later is 151/90 mm Hg. The fetal heart rate is 151/min. The physical examination is significant for 2+ pitting edema of the lower extremity.
Question: Which of the following tests should confirm the probable condition of this patient? Options: {A: Bilirubin assessment, B: Coagulation studies, C: Hematocrit assessment, D: Leukocyte count with differential, E: 24-hour urine protein}
Answer:
"""
med_output7 = med_pipeline(response6)
med_output7

[{'generated_text': 'Answer the question using the context below.\nContext: A 25-year-old primigravida presents to her physician for a routine prenatal visit. She is at 34 weeks gestation, as confirmed by an ultrasound examination. She has no complaints, but notes that the new shoes she bought 2 weeks ago do not fit anymore. The course of her pregnancy has been uneventful and she has been compliant with the recommended prenatal care. Her medical history is unremarkable. She has a 15-pound weight gain since the last visit 3 weeks ago. Her vital signs are as follows: blood pressure, 148/90 mm Hg; heart rate, 88/min; respiratory rate, 16/min; and temperature, 36.6℃ (97.9℉). The blood pressure on repeat assessment 4 hours later is 151/90 mm Hg. The fetal heart rate is 151/min. The physical examination is significant for 2+ pitting edema of the lower extremity.\nQuestion: Which of the following tests should confirm the probable condition of this patient? Options: {A: Bilirubin assessment, B

In [15]:
response7 = """Answer the question using the context below.
Context: A 3900-g (8.6-lb) male infant is delivered at 39 weeks' gestation via spontaneous vaginal delivery. Pregnancy and delivery were uncomplicated but a prenatal ultrasound at 20 weeks showed a defect in the pleuroperitoneal membrane. Further evaluation of this patient is most likely to show which of the following findings?
Question: Options: {A: Gastric fundus in the thorax, B: Pancreatic ring around the duodenum, C: Small and cystic kidneys, D: Hypertrophy of the gastric pylorus, E: Large bowel in the inguinal canal}
Answer:
"""
med_output8 = med_pipeline(response7)
med_output8

[{'generated_text': "Answer the question using the context below.\nContext: A 3900-g (8.6-lb) male infant is delivered at 39 weeks' gestation via spontaneous vaginal delivery. Pregnancy and delivery were uncomplicated but a prenatal ultrasound at 20 weeks showed a defect in the pleuroperitoneal membrane. Further evaluation of this patient is most likely to show which of the following findings?\nQuestion: Options: {A: Gastric fundus in the thorax, B: Pancreatic ring around the duodenum, C: Small and cystic kidneys, D: Hypertrophy of the gastric pylorus, E: Large bowel in the inguinal canal}\nAnswer:\nA"}]

In [16]:
response8 = """Answer the question using the context below.
Context: A 62-year-old woman presents for a regular check-up. She complains of lightheadedness and palpitations which occur episodically. Past medical history is significant for a myocardial infarction 6 months ago and NYHA class II chronic heart failure. She also was diagnosed with grade I arterial hypertension 4 years ago. Current medications are aspirin 81 mg, atorvastatin 10 mg, enalapril 10 mg, and metoprolol 200 mg daily. Her vital signs are a blood pressure of 135/90 mm Hg, a heart rate of 125/min, a respiratory rate of 14/min, and a temperature of 36.5°C (97.7°F). Cardiopulmonary examination is significant for irregular heart rhythm and decreased S1 intensity. ECG is obtained and is shown in the picture (see image). Echocardiography shows a left ventricular ejection fraction of 39%.
Question: Which of the following drugs is the best choice for rate control in this patient? Options: {A: Atenolol, B: Verapamil, C: Diltiazem, D: Propafenone, E: Digoxin}
Answer:
"""
med_output9 = med_pipeline(response8)
med_output9

[{'generated_text': 'Answer the question using the context below.\nContext: A 62-year-old woman presents for a regular check-up. She complains of lightheadedness and palpitations which occur episodically. Past medical history is significant for a myocardial infarction 6 months ago and NYHA class II chronic heart failure. She also was diagnosed with grade I arterial hypertension 4 years ago. Current medications are aspirin 81 mg, atorvastatin 10 mg, enalapril 10 mg, and metoprolol 200 mg daily. Her vital signs are a blood pressure of 135/90 mm Hg, a heart rate of 125/min, a respiratory rate of 14/min, and a temperature of 36.5°C (97.7°F). Cardiopulmonary examination is significant for irregular heart rhythm and decreased S1 intensity. ECG is obtained and is shown in the picture (see image). Echocardiography shows a left ventricular ejection fraction of 39%.\nQuestion: Which of the following drugs is the best choice for rate control in this patient? Options: {A: Atenolol, B: Verapamil, C

In [17]:
response9 = """Answer the question using the context below.
Context: A 35-year-old male presents to his primary care physician with complaints of seasonal allergies. He has been using intranasal vasoconstrictors several times per day for several weeks.
Question: What is a likely sequela of the chronic use of topical nasal decongestants? Options: {A: Epistaxis, B: Hypertension, C: Permanent loss of smell, D: Persistent nasal crusting, E: Persistent congestion}
Answer:
"""
med_output10 = med_pipeline(response9)
med_output10

[{'generated_text': 'Answer the question using the context below.\nContext: A 35-year-old male presents to his primary care physician with complaints of seasonal allergies. He has been using intranasal vasoconstrictors several times per day for several weeks.\nQuestion: What is a likely sequela of the chronic use of topical nasal decongestants? Options: {A: Epistaxis, B: Hypertension, C: Permanent loss of smell, D: Persistent nasal crusting, E: Persistent congestion}\nAnswer:\nA'}]

In [18]:
response10 = """Answer the question using the context below.
Context: A 46-year-old woman comes to the physician because of a 2-week history of diplopia and ocular pain when reading the newspaper. She also has a 3-month history of amenorrhea, hot flashes, and increased sweating. She reports that she has been overweight all her adult life and is happy to have lost 6.8-kg (15-lb) of weight in the past 2 months. Her pulse is 110/min, and blood pressure is 148/98 mm Hg. Physical examination shows moist palms and a nontender thyroid gland that is enlarged to two times its normal size. Ophthalmologic examination shows prominence of the globes of the eyes, bilateral lid retraction, conjunctival injection, and an inability to converge the eyes. There is no pain on movement of the extraocular muscles. Visual acuity is 20/20 bilaterally. Neurologic examination shows a fine resting tremor of the hands. Deep tendon reflexes are 3+ with a shortened relaxation phase.
Question: Which of the following is the most likely cause of this patient's ocular complaints? Options: {A: Granulomatous inflammation of the cavernous sinus, B: Abnormal communication between the cavernous sinus and the internal carotid artery, C: Glycosaminoglycan accumulation in the orbit, D: Bacterial infection of the orbital contents, E: Sympathetic hyperactivity of levator palpebrae superioris}
Answer:
"""
med_output11 = med_pipeline(response10) 
med_output11

[{'generated_text': "Answer the question using the context below.\nContext: A 46-year-old woman comes to the physician because of a 2-week history of diplopia and ocular pain when reading the newspaper. She also has a 3-month history of amenorrhea, hot flashes, and increased sweating. She reports that she has been overweight all her adult life and is happy to have lost 6.8-kg (15-lb) of weight in the past 2 months. Her pulse is 110/min, and blood pressure is 148/98 mm Hg. Physical examination shows moist palms and a nontender thyroid gland that is enlarged to two times its normal size. Ophthalmologic examination shows prominence of the globes of the eyes, bilateral lid retraction, conjunctival injection, and an inability to converge the eyes. There is no pain on movement of the extraocular muscles. Visual acuity is 20/20 bilaterally. Neurologic examination shows a fine resting tremor of the hands. Deep tendon reflexes are 3+ with a shortened relaxation phase.\nQuestion: Which of the fo

In [19]:
response11 = """Answer the question using the context below.
Context: A 51-year-old woman comes to the physician because of a 1-day history of right flank pain and bloody urine. Over the past 2 weeks, she has also developed progressive lower extremity swelling and a 3-kg (7-lb) weight gain. She has a history of chronic hepatitis B infection, which was diagnosed 10 years ago. She frequently flies from California to New York for business. She appears fatigued. Her pulse is 98/min, respirations are 18/min, and blood pressure is 135/75 mm Hg. Examination shows periorbital edema, a distended abdomen, and 2+ edema of the lower extremities. The lungs are clear to auscultation. A CT scan of the abdomen shows a nodular liver with ascites, a large right kidney with abundant collateral vessels, and a filling defect in the right renal vein. Urinalysis shows 4+ protein, positive glucose, and fatty casts.
Question: Which of the following is the most likely underlying cause of this patient's renal vein findings? Options: {A: "Acquired factor VIII deficiency", B: "Loss of antithrombin III", C: "Impaired estrogen degradation", D: "Antiphospholipid antibodies", E: "Paraneoplastic erythropoietin production"}
Answer:
"""
med_output12 = med_pipeline(response11)
med_output12

[{'generated_text': 'Answer the question using the context below.\nContext: A 51-year-old woman comes to the physician because of a 1-day history of right flank pain and bloody urine. Over the past 2 weeks, she has also developed progressive lower extremity swelling and a 3-kg (7-lb) weight gain. She has a history of chronic hepatitis B infection, which was diagnosed 10 years ago. She frequently flies from California to New York for business. She appears fatigued. Her pulse is 98/min, respirations are 18/min, and blood pressure is 135/75 mm Hg. Examination shows periorbital edema, a distended abdomen, and 2+ edema of the lower extremities. The lungs are clear to auscultation. A CT scan of the abdomen shows a nodular liver with ascites, a large right kidney with abundant collateral vessels, and a filling defect in the right renal vein. Urinalysis shows 4+ protein, positive glucose, and fatty casts.\nQuestion: Which of the following is the most likely underlying cause of this patient\'s 

In [20]:
response12 = """Answer the question using the context below.
Context: A 57-year-old man comes to the physician for a follow-up evaluation of chronic, retrosternal chest pain. The pain is worse at night and after heavy meals. He has taken oral pantoprazole for several months without any relief of his symptoms. Esophagogastroduodenoscopy shows ulcerations in the distal esophagus and a proximally dislocated Z-line. A biopsy of the distal esophagus shows columnar epithelium with goblet cells.
Question: Which of the following microscopic findings underlie the same pathomechanism as the cellular changes seen in this patient? Options: {A: Pseudostratified columnar epithelium in the bronchi, B: Squamous epithelium in the bladder, C: Paneth cells in the duodenum, D: Branching muscularis mucosa in the jejunum, E: Disorganized squamous epithelium in the endocervix}
Answer:
"""
med_output13 = med_pipeline(response12)
med_output13

[{'generated_text': 'Answer the question using the context below.\nContext: A 57-year-old man comes to the physician for a follow-up evaluation of chronic, retrosternal chest pain. The pain is worse at night and after heavy meals. He has taken oral pantoprazole for several months without any relief of his symptoms. Esophagogastroduodenoscopy shows ulcerations in the distal esophagus and a proximally dislocated Z-line. A biopsy of the distal esophagus shows columnar epithelium with goblet cells.\nQuestion: Which of the following microscopic findings underlie the same pathomechanism as the cellular changes seen in this patient? Options: {A: Pseudostratified columnar epithelium in the bronchi, B: Squamous epithelium in the bladder, C: Paneth cells in the duodenum, D: Branching muscularis mucosa in the jejunum, E: Disorganized squamous epithelium in the endocervix}\nAnswer:\nC'}]

In [21]:
response13 = """Answer the question using the context below.
Context: A 37-year-old woman comes to the physician because of a 6-month history of weight loss, bloating, and diarrhea. She does not smoke or drink alcohol. Her vital signs are within normal limits. She is 173 cm (5 ft 8 in) tall and weighs 54 kg (120 lb); BMI is 18 kg/m2. Physical examination shows bilateral white spots on the temporal half of the conjunctiva, dry skin, and a hard neck mass in the anterior midline that does not move with swallowing. Urinalysis after a D-xylose meal shows an increase in renal D-xylose excretion.
Question: Which of the following is most likely to have prevented this patient's weight loss? Options: {A: Gluten-free diet, B: Pancreatic enzyme replacement, C: Tetracycline therapy, D: Mesalamine therapy, E: Lactose-free diet}
Answer:
"""
med_output14 = med_pipeline(response13)
med_output14

[{'generated_text': "Answer the question using the context below.\nContext: A 37-year-old woman comes to the physician because of a 6-month history of weight loss, bloating, and diarrhea. She does not smoke or drink alcohol. Her vital signs are within normal limits. She is 173 cm (5 ft 8 in) tall and weighs 54 kg (120 lb); BMI is 18 kg/m2. Physical examination shows bilateral white spots on the temporal half of the conjunctiva, dry skin, and a hard neck mass in the anterior midline that does not move with swallowing. Urinalysis after a D-xylose meal shows an increase in renal D-xylose excretion.\nQuestion: Which of the following is most likely to have prevented this patient's weight loss? Options: {A: Gluten-free diet, B: Pancreatic enzyme replacement, C: Tetracycline therapy, D: Mesalamine therapy, E: Lactose-free diet}\nAnswer:\nC"}]

In [22]:
response14 = """Answer the question using the context below.
Context: A 52-year-old man presents for a routine checkup. Past medical history is remarkable for stage 1 systemic hypertension and hepatitis A infection diagnosed 10 years ago. He takes aspirin, rosuvastatin, enalapril daily, and a magnesium supplement every once in a while. He is planning to visit Ecuador for a week-long vacation and is concerned about malaria prophylaxis before his travel. The physician advised taking 1 primaquine pill every day while he is there and for 7 consecutive days after leaving Ecuador. On the third day of his trip, the patient develops an acute onset headache, dizziness, shortness of breath, and fingertips and toes turning blue. His blood pressure is 135/80 mm Hg, heart rate is 94/min, respiratory rate is 22/min, temperature is 36.9℃ (98.4℉), and blood oxygen saturation is 97% in room air. While drawing blood for his laboratory workup, the nurse notes that his blood has a chocolate brown color.
Question: Which of the following statements best describes the etiology of this patient’s most likely condition? Options: {A: The patient’s condition is due to consumption of water polluted with nitrates., B: This condition resulted from primaquine overdose., C: The patient had pre-existing liver damage caused by viral hepatitis., D: The condition developed because of his concomitant use of primaquine and magnesium supplement., E: It is a type B adverse drug reaction.}
Answer:
"""
med_output15 = med_pipeline(response14)
med_output15

[{'generated_text': 'Answer the question using the context below.\nContext: A 52-year-old man presents for a routine checkup. Past medical history is remarkable for stage 1 systemic hypertension and hepatitis A infection diagnosed 10 years ago. He takes aspirin, rosuvastatin, enalapril daily, and a magnesium supplement every once in a while. He is planning to visit Ecuador for a week-long vacation and is concerned about malaria prophylaxis before his travel. The physician advised taking 1 primaquine pill every day while he is there and for 7 consecutive days after leaving Ecuador. On the third day of his trip, the patient develops an acute onset headache, dizziness, shortness of breath, and fingertips and toes turning blue. His blood pressure is 135/80 mm Hg, heart rate is 94/min, respiratory rate is 22/min, temperature is 36.9℃ (98.4℉), and blood oxygen saturation is 97% in room air. While drawing blood for his laboratory workup, the nurse notes that his blood has a chocolate brown co

In [23]:
response15 = """Answer the question using the context below.
Context: A 31-year-old woman, gravida 2, para 1, at 32 weeks' gestation comes to the emergency department for sudden leakage of clear vaginal fluid. Her pregnancy has been uncomplicated. Her first child was born at term by vaginal delivery. She has no history of serious illness. She does not drink alcohol or smoke cigarettes. Current medications include vitamin supplements. Her temperature is 37.2°C (98.9°F), pulse is 70/min, respirations are 18/min, and blood pressure is 128/82 mm Hg. Speculum examination demonstrates clear fluid in the cervical canal. The fetal heart rate is reactive at 160/min with no decelerations. Tocometry shows uterine contractions. Nitrazine testing is positive. She is started on indomethacin.
Question: Which of the following is the most appropriate next step in management? Options: {A: Administer betamethasone, ampicillin, and proceed with cesarean section, B: Administer ampicillin and perform amnioinfusion, C: Administer ampicillin and test amniotic fluid for fetal lung maturity, D: Administer betamethasone and ampicillin, E: Administer betamethasone, ampicillin, and proceed with induction of labor}
Answer:
"""
med_output16 = med_pipeline(response15)
med_output16

[{'generated_text': "Answer the question using the context below.\nContext: A 31-year-old woman, gravida 2, para 1, at 32 weeks' gestation comes to the emergency department for sudden leakage of clear vaginal fluid. Her pregnancy has been uncomplicated. Her first child was born at term by vaginal delivery. She has no history of serious illness. She does not drink alcohol or smoke cigarettes. Current medications include vitamin supplements. Her temperature is 37.2°C (98.9°F), pulse is 70/min, respirations are 18/min, and blood pressure is 128/82 mm Hg. Speculum examination demonstrates clear fluid in the cervical canal. The fetal heart rate is reactive at 160/min with no decelerations. Tocometry shows uterine contractions. Nitrazine testing is positive. She is started on indomethacin.\nQuestion: Which of the following is the most appropriate next step in management? Options: {A: Administer betamethasone, ampicillin, and proceed with cesarean section, B: Administer ampicillin and perform

In [24]:
response16 = """Answer the question using the context below.
Context: A 16-year-old girl is brought to the emergency department by her friends who say that she took a whole bottle of her mom’s medication. They do not know which medication it was she ingested. The patient is slipping in and out of consciousness and is unable to offer any history. Her temperature is 39.6°C (103.2°F), the heart rate is 135/min, the blood pressure is 178/98 mm Hg, and the respiratory rate is 16/min. On physical examination, there is significant muscle rigidity without tremor or clonus.
Question: Which of the following is the best course of treatment for this patient? Options: {A: Naloxone, B: Dantrolene, C: Fenoldopam, D: Cyproheptadine, E: Flumazenil}
Answer:
"""
med_output17 = med_pipeline(response16)
med_output17

[{'generated_text': 'Answer the question using the context below.\nContext: A 16-year-old girl is brought to the emergency department by her friends who say that she took a whole bottle of her mom’s medication. They do not know which medication it was she ingested. The patient is slipping in and out of consciousness and is unable to offer any history. Her temperature is 39.6°C (103.2°F), the heart rate is 135/min, the blood pressure is 178/98 mm Hg, and the respiratory rate is 16/min. On physical examination, there is significant muscle rigidity without tremor or clonus.\nQuestion: Which of the following is the best course of treatment for this patient? Options: {A: Naloxone, B: Dantrolene, C: Fenoldopam, D: Cyproheptadine, E: Flumazenil}\nAnswer:\nA'}]

In [25]:
response17 = """Answer the question using the context below.
Context: A 68-year-old woman is brought to the emergency department because of fever, productive cough, and dyspnea for 3 days. She has had upper back pain for 3 months, which is worse after activity. She takes ibuprofen for pain relief. She has no history of smoking. The temperature is 39.5°C (103.1°F), the blood pressure is 100/70 mm Hg, the pulse is 95/min, and the respirations are 22/min. Lung auscultation shows rales in the left lower lobe area. Painful lymph nodes (1 × 1 cm) are palpated in the left axillary and cervical regions. There is point tenderness along several thoracic vertebrae. Laboratory studies are pending. A skull X-ray and lung window thoracic computed tomography scan are shown.
Question: Which of the following disorders most likely played a role in this patient’s acute condition? Options: {A: Metastatic breast cancer, B: Multiple myeloma, C: Non-small cell lung cancer, D: Paget’s disease, E: Primary hyperparathyroidism}
Answer:
"""
med_output18 = med_pipeline(response17)
med_output18

[{'generated_text': 'Answer the question using the context below.\nContext: A 68-year-old woman is brought to the emergency department because of fever, productive cough, and dyspnea for 3 days. She has had upper back pain for 3 months, which is worse after activity. She takes ibuprofen for pain relief. She has no history of smoking. The temperature is 39.5°C (103.1°F), the blood pressure is 100/70 mm Hg, the pulse is 95/min, and the respirations are 22/min. Lung auscultation shows rales in the left lower lobe area. Painful lymph nodes (1 × 1 cm) are palpated in the left axillary and cervical regions. There is point tenderness along several thoracic vertebrae. Laboratory studies are pending. A skull X-ray and lung window thoracic computed tomography scan are shown.\nQuestion: Which of the following disorders most likely played a role in this patient’s acute condition? Options: {A: Metastatic breast cancer, B: Multiple myeloma, C: Non-small cell lung cancer, D: Paget’s disease, E: Prima

In [26]:
response18 = """Answer the question using the context below.
Context: A 22-year-old woman presents to the emergency department with a 2-day history of severe blistering. She says that she woke up 2 days ago with a number of painful blisters in her mouth and has since been continuing to develop blisters of her cutaneous skin all over her body and the mucosa of her mouth. She has no past medical history and has never experienced these symptoms before. Physical exam reveals a diffuse vesicular rash with painful, flaccid blisters that separate easily with gentle rubbing.
Question: The function of which of the following proteins is most likely disrupted in this patient? Options: {A: Cadherin, B: Collagen, C: Integrin, D: Keratin, E: T-cell receptor}
Answer:
"""
med_output19 = med_pipeline(response18)
med_output19

[{'generated_text': 'Answer the question using the context below.\nContext: A 22-year-old woman presents to the emergency department with a 2-day history of severe blistering. She says that she woke up 2 days ago with a number of painful blisters in her mouth and has since been continuing to develop blisters of her cutaneous skin all over her body and the mucosa of her mouth. She has no past medical history and has never experienced these symptoms before. Physical exam reveals a diffuse vesicular rash with painful, flaccid blisters that separate easily with gentle rubbing.\nQuestion: The function of which of the following proteins is most likely disrupted in this patient? Options: {A: Cadherin, B: Collagen, C: Integrin, D: Keratin, E: T-cell receptor}\nAnswer:\nC'}]

In [27]:
response19 = """Answer the question using the context below.
Context: A 3-week-old boy is brought to the emergency department by his parents because of a 3-day history of progressive lethargy and difficulty feeding. He was born at term and did not have difficulty feeding previously. His temperature is 39.4°C (103°F), pulse is 220/min, respirations are 45/min, and blood pressure is 50/30 mm Hg. Pulse oximetry on 100% oxygen shows an oxygen saturation of 97%. Examination shows dry mucous membranes, delayed capillary refill time, and cool skin with poor turgor. Despite multiple attempts by the nursing staff, they are unable to establish peripheral intravenous access.
Question: Which of the following is the most appropriate next step in management? Options: {A: Rapid sequence intubation, B: Intramuscular epinephrine, C: Internal jugular vein cannulation, D: Intraosseous cannulation, E: Ultrasound-guided antecubital vein cannulation}
Answer:
"""
med_output20 = med_pipeline(response19)
med_output20

[{'generated_text': 'Answer the question using the context below.\nContext: A 3-week-old boy is brought to the emergency department by his parents because of a 3-day history of progressive lethargy and difficulty feeding. He was born at term and did not have difficulty feeding previously. His temperature is 39.4°C (103°F), pulse is 220/min, respirations are 45/min, and blood pressure is 50/30 mm Hg. Pulse oximetry on 100% oxygen shows an oxygen saturation of 97%. Examination shows dry mucous membranes, delayed capillary refill time, and cool skin with poor turgor. Despite multiple attempts by the nursing staff, they are unable to establish peripheral intravenous access.\nQuestion: Which of the following is the most appropriate next step in management? Options: {A: Rapid sequence intubation, B: Intramuscular epinephrine, C: Internal jugular vein cannulation, D: Intraosseous cannulation, E: Ultrasound-guided antecubital vein cannulation}\nAnswer:\nD'}]

In [28]:
response20 = """Answer the question using the context below.
Context: A previously healthy 10-year-old boy is brought to the emergency room by his mother 5 hours after the onset of abdominal pain and nausea. Over the past 2 weeks, he has also had progressive abdominal pain and a 4-kg (8.8-lb) weight loss. The mother reports that her son has been drinking more water than usual during this period. Last week he wet his bed three times despite being completely toilet-trained since 3 years of age. His temperature is 37.8°C (100°F), pulse is 128/min, respirations are 35/min, and blood pressure is 95/55 mm Hg. He appears lethargic. Physical examination shows deep and labored breathing and dry mucous membranes. The abdomen is soft, and there is diffuse tenderness to palpation with no guarding or rebound. Serum laboratory studies show:
Na+ 133 mEq/L
K+ 5.9 mEq/L
Cl- 95 mEq/L
HCO3- 13 mEq/L
Urea nitrogen 25 mg/dL
Creatinine 1.0 mg/dL
Urine dipstick is positive for ketones and glucose.
Question: Further evaluation is most likely to reveal which of the following? Options: {A: Decreased total body potassium, B: Increased total body sodium, C: Increased arterial pCO2, D: Hypervolemia, E: Serum glucose concentration > 600 mg/dL}
Answer:
"""
med_output21 = med_pipeline(response20)
med_output21

[{'generated_text': 'Answer the question using the context below.\nContext: A previously healthy 10-year-old boy is brought to the emergency room by his mother 5 hours after the onset of abdominal pain and nausea. Over the past 2 weeks, he has also had progressive abdominal pain and a 4-kg (8.8-lb) weight loss. The mother reports that her son has been drinking more water than usual during this period. Last week he wet his bed three times despite being completely toilet-trained since 3 years of age. His temperature is 37.8°C (100°F), pulse is 128/min, respirations are 35/min, and blood pressure is 95/55 mm Hg. He appears lethargic. Physical examination shows deep and labored breathing and dry mucous membranes. The abdomen is soft, and there is diffuse tenderness to palpation with no guarding or rebound. Serum laboratory studies show:\nNa+ 133 mEq/L\nK+ 5.9 mEq/L\nCl- 95 mEq/L\nHCO3- 13 mEq/L\nUrea nitrogen 25 mg/dL\nCreatinine 1.0 mg/dL\nUrine dipstick is positive for ketones and glucos

In [29]:
response21 = """Answer the question using the context below.
Context: A 70-year-old Caucasian male visits your office regularly for treatment of New York Heart association class IV congestive heart failure.
Question: Which of the following medications would you add to this man's drug regimen in order to improve his overall survival? Options: {A: Spironolactone, B: Amiloride, C: Hydrochlorothiazide, D: Furosemide, E: Acetazolamide}
Answer:
"""
med_output22 = med_pipeline(response21)
med_output22

[{'generated_text': "Answer the question using the context below.\nContext: A 70-year-old Caucasian male visits your office regularly for treatment of New York Heart association class IV congestive heart failure.\nQuestion: Which of the following medications would you add to this man's drug regimen in order to improve his overall survival? Options: {A: Spironolactone, B: Amiloride, C: Hydrochlorothiazide, D: Furosemide, E: Acetazolamide}\nAnswer:\nA"}]

In [30]:
response22 = """Answer the question using the context below.
Context: Several hours after vaginal delivery, a male newborn delivered at full-term develops tachycardia and tachypnea. His blood pressure is within normal limits. Pulse oximetry on room air shows an oxygen saturation of 79% in the right hand and 61% in the left foot. Physical examination shows bluish discoloration of the face and trunk, supraclavicular and intercostal retractions, and a machine-like murmur over the precordium. Bedside echocardiography shows pulmonary and systemic circulation are in parallel rather than in series.
Question: What is the most appropriate pharmacotherapy for this patient? Options: {A: Sildenafil, B: Alprostadil, C: Metoprolol, D: Indomethacin, E: Dopamine}
Answer:
"""
med_output23 = med_pipeline(response22)
med_output23

[{'generated_text': 'Answer the question using the context below.\nContext: Several hours after vaginal delivery, a male newborn delivered at full-term develops tachycardia and tachypnea. His blood pressure is within normal limits. Pulse oximetry on room air shows an oxygen saturation of 79% in the right hand and 61% in the left foot. Physical examination shows bluish discoloration of the face and trunk, supraclavicular and intercostal retractions, and a machine-like murmur over the precordium. Bedside echocardiography shows pulmonary and systemic circulation are in parallel rather than in series.\nQuestion: What is the most appropriate pharmacotherapy for this patient? Options: {A: Sildenafil, B: Alprostadil, C: Metoprolol, D: Indomethacin, E: Dopamine}\nAnswer:\nA'}]

In [31]:
response23 = """Answer the question using the context below.
Context: A 5-year-old male visits his pediatrician for a check-up. His height corresponds to the 99th percentile for his age, and pubic hair is present upon physical examination. Serum renin and potassium levels are high, as is 17-hydroxyprogesterone.
Question: Which of the following is likely deficient in this patient? Options: {A: 17a-hydroxylase, B: 11ß-hydroxylase, C: 21-hydroxylase, D: Aromatase, E: 5a-reductase}
Answer:
"""
med_output24 = med_pipeline(response23)
med_output24

[{'generated_text': 'Answer the question using the context below.\nContext: A 5-year-old male visits his pediatrician for a check-up. His height corresponds to the 99th percentile for his age, and pubic hair is present upon physical examination. Serum renin and potassium levels are high, as is 17-hydroxyprogesterone.\nQuestion: Which of the following is likely deficient in this patient? Options: {A: 17a-hydroxylase, B: 11ß-hydroxylase, C: 21-hydroxylase, D: Aromatase, E: 5a-reductase}\nAnswer:\nC'}]

In [32]:
response24 = """Answer the question using the context below.
Context: A 41-year-old African American woman presents with her husband to her primary care doctor for evaluation of depression and anxiety. She reports a 2-week history of rapid onset sadness with no clear inciting factor. She is accompanied by her husband who notes that she has had at least three similar episodes that have occurred over the past two years. He also notes that she has been “more emotional” lately and seems confused throughout the day. She has had to leave her job as a librarian at her child’s elementary school. Her past medical history is notable for two diagnostic laparoscopies for recurrent episodes of abdominal pain of unknown etiology. Her family history is notable for psychosis in her mother and maternal grandfather. Her temperature is 99°F (37.2°C), blood pressure is 125/75 mmHg, pulse is 75/min, and respirations are 17/min. On exam, she is disheveled and appears confused and disoriented. Her attention span is limited and she exhibits emotional lability.
QuestionThis patient’s condition is most likely due to a defect in an enzyme that metabolizes which of the following compounds? Options: {A: Aminolevulinic acid, B: Coproporphyrinogen III, C: Hydroxymethylbilane, D: Porphobilinogen, E: Protoporphyrin IX}
Answer:
"""
med_output25 = med_pipeline(response24)
med_output25

[{'generated_text': 'Answer the question using the context below.\nContext: A 41-year-old African American woman presents with her husband to her primary care doctor for evaluation of depression and anxiety. She reports a 2-week history of rapid onset sadness with no clear inciting factor. She is accompanied by her husband who notes that she has had at least three similar episodes that have occurred over the past two years. He also notes that she has been “more emotional” lately and seems confused throughout the day. She has had to leave her job as a librarian at her child’s elementary school. Her past medical history is notable for two diagnostic laparoscopies for recurrent episodes of abdominal pain of unknown etiology. Her family history is notable for psychosis in her mother and maternal grandfather. Her temperature is 99°F (37.2°C), blood pressure is 125/75 mmHg, pulse is 75/min, and respirations are 17/min. On exam, she is disheveled and appears confused and disoriented. Her atte

In [33]:
response25 = """Answer the question using the context below.
Context: A 32-year-old woman presents to her primary care physician for a general wellness appointment. The patient has no complaints currently and just wants to be sure that she is in good health. The patient has a past medical history of asthma, hypertension, and anxiety. Her current medications include albuterol, fluticasone, hydrochlorothiazide, lisinopril, and fexofenadine. Her temperature is 99.5°F (37.5°C), blood pressure is 165/95 mmHg, pulse is 70/min, respirations are 15/min, and oxygen saturation is 98% on room air. On exam, you note a healthy young woman with a lean habitus. Cardiac exam reveals a S1 and S2 heart sound with a normal rate. Pulmonary exam is clear to auscultation bilaterally with good air movement. Abdominal exam reveals a bruit, normoactive bowel sounds, and an audible borborygmus. Neurological exam reveals cranial nerves II-XII as grossly intact with normal strength and reflexes in the upper and lower extremities.
Question: Which of the following is the best next step in management? Options: {A: Raise lisinopril dose, B: Add furosemide, C: Ultrasound with doppler, D: CT of the abdomen, E: No additional management needed}
Answer:
"""
med_output26 = med_pipeline(response25)
med_output26


[{'generated_text': 'Answer the question using the context below.\nContext: A 32-year-old woman presents to her primary care physician for a general wellness appointment. The patient has no complaints currently and just wants to be sure that she is in good health. The patient has a past medical history of asthma, hypertension, and anxiety. Her current medications include albuterol, fluticasone, hydrochlorothiazide, lisinopril, and fexofenadine. Her temperature is 99.5°F (37.5°C), blood pressure is 165/95 mmHg, pulse is 70/min, respirations are 15/min, and oxygen saturation is 98% on room air. On exam, you note a healthy young woman with a lean habitus. Cardiac exam reveals a S1 and S2 heart sound with a normal rate. Pulmonary exam is clear to auscultation bilaterally with good air movement. Abdominal exam reveals a bruit, normoactive bowel sounds, and an audible borborygmus. Neurological exam reveals cranial nerves II-XII as grossly intact with normal strength and reflexes in the upper

In [34]:
response26 = """Answer the question using the context below.
Context: A 46-year-old man comes to the emergency department because of a 10-day history of right upper quadrant abdominal pain. He has also been feeling tired and nauseous for the past 6 weeks. On examination, scleral icterus is present. Abdominal examination shows tenderness to palpation in the right upper quadrant. The liver edge is palpated 2 cm below the right costal margin. Laboratory studies show:
Aspartate aminotransferase 1780 U/L
Alanine aminotransferase 2520 U/L
Hepatitis A IgM antibody Negative
Hepatitis B surface antigen Negative
Hepatitis B surface antibody Negative
Hepatitis B core IgM antibody Positive
Hepatitis C antibody Positive
Hepatitis C RNA Negative
Question: Which of the following is the best course of action for this patient? Options: {A: Ribavirin and interferon, B: Supportive therapy, C: Tenofovir, D: Emergency liver transplantation, E: Pegylated interferon-alpha}
Answer:
"""
med_output27 = med_pipeline(response26)
med_output27

[{'generated_text': 'Answer the question using the context below.\nContext: A 46-year-old man comes to the emergency department because of a 10-day history of right upper quadrant abdominal pain. He has also been feeling tired and nauseous for the past 6 weeks. On examination, scleral icterus is present. Abdominal examination shows tenderness to palpation in the right upper quadrant. The liver edge is palpated 2 cm below the right costal margin. Laboratory studies show:\nAspartate aminotransferase 1780 U/L\nAlanine aminotransferase 2520 U/L\nHepatitis A IgM antibody Negative\nHepatitis B surface antigen Negative\nHepatitis B surface antibody Negative\nHepatitis B core IgM antibody Positive\nHepatitis C antibody Positive\nHepatitis C RNA Negative\nQuestion: Which of the following is the best course of action for this patient? Options: {A: Ribavirin and interferon, B: Supportive therapy, C: Tenofovir, D: Emergency liver transplantation, E: Pegylated interferon-alpha}\nAnswer:\nB'}]

In [35]:
response27 = """Answer the question using the context below.
Context: A 5-year-old boy who recently emigrated from Nigeria is brought to the emergency department because of a 2-day history of lower leg weakness, swallowing difficulty, and drooling of saliva. He has not yet received any childhood vaccinations. Two days after admission, the patient develops shortness of breath. Pulse oximetry shows an oxygen saturation of 64%. Despite resuscitative efforts, the patient dies of respiratory failure. At autopsy, examination of the spinal cord shows destruction of the anterior horn cells.
Question: Neurological examination of this patient would have most likely shown which of the following findings? Options: {A: Positive Babinski sign, B: Hyporeflexia, C: Sensory loss, D: Myoclonus, E: Pronator drift}
Answer:
"""
med_output28 = med_pipeline(response27)
med_output28

[{'generated_text': 'Answer the question using the context below.\nContext: A 5-year-old boy who recently emigrated from Nigeria is brought to the emergency department because of a 2-day history of lower leg weakness, swallowing difficulty, and drooling of saliva. He has not yet received any childhood vaccinations. Two days after admission, the patient develops shortness of breath. Pulse oximetry shows an oxygen saturation of 64%. Despite resuscitative efforts, the patient dies of respiratory failure. At autopsy, examination of the spinal cord shows destruction of the anterior horn cells.\nQuestion: Neurological examination of this patient would have most likely shown which of the following findings? Options: {A: Positive Babinski sign, B: Hyporeflexia, C: Sensory loss, D: Myoclonus, E: Pronator drift}\nAnswer:\nA'}]

In [38]:
response28 = """Answer the question using the context below.
Context: A woman with coronary artery disease is starting to go for a walk. As she begins, her heart rate accelerates from a resting pulse of 60 bpm until it reaches a rate of 120 bpm, at which point she begins to feel a tightening in her chest. She stops walking to rest and the tightening resolves. This has been happening to her consistently for the last 6 months.
Question: Which of the following is a true statement? Options: {A: Increasing the heart rate increases the amount of time spent during each cardiac cycle, B: Increasing the heart rate decreases the relative amount of time spent during diastole, C: Perfusion of the myocardium takes place primarily during systole, D: Perfusion of the myocardium takes place equally throughout the cardiac cycle, E: This patient's chest pain is indicative of transmural ischemia}
Answer:
"""
med_output29 = med_pipeline(response28)
med_output29

[{'generated_text': "Answer the question using the context below.\nContext: A woman with coronary artery disease is starting to go for a walk. As she begins, her heart rate accelerates from a resting pulse of 60 bpm until it reaches a rate of 120 bpm, at which point she begins to feel a tightening in her chest. She stops walking to rest and the tightening resolves. This has been happening to her consistently for the last 6 months.\nQuestion: Which of the following is a true statement? Options: {A: Increasing the heart rate increases the amount of time spent during each cardiac cycle, B: Increasing the heart rate decreases the relative amount of time spent during diastole, C: Perfusion of the myocardium takes place primarily during systole, D: Perfusion of the myocardium takes place equally throughout the cardiac cycle, E: This patient's chest pain is indicative of transmural ischemia}\nAnswer:\nB"}]

In [39]:
response29 = """Answer the question using the context below.
Context: A 22-year-old female presents to her physician for evaluation of a vaginal discharge, itching, and irritation. She recently started a new relationship with her boyfriend, who is her only sexual partner. He does not report any genitourinary symptoms. She takes oral contraceptives and does not use barrier contraception. The medical history is unremarkable. The vital signs are within normal limits. A gynecologic examination reveals a thin, yellow, frothy vaginal discharge with a musty, unpleasant odor and numerous punctate red maculae on the ectocervix. The remainder of the exam is normal.
Question: Which of the following organisms will most likely be revealed on wet mount microscopy? Options: {A: Budding yeasts cells and/or pseudohyphae, B: Epithelial cells covered by numerous bacterial cells, C: Motile round or oval-shaped microorganisms, D: Numerous rod-shaped bacteria, E: Chains of cocci}
Answer:
"""
med_output30 = med_pipeline(response29)
med_output30

[{'generated_text': 'Answer the question using the context below.\nContext: A 22-year-old female presents to her physician for evaluation of a vaginal discharge, itching, and irritation. She recently started a new relationship with her boyfriend, who is her only sexual partner. He does not report any genitourinary symptoms. She takes oral contraceptives and does not use barrier contraception. The medical history is unremarkable. The vital signs are within normal limits. A gynecologic examination reveals a thin, yellow, frothy vaginal discharge with a musty, unpleasant odor and numerous punctate red maculae on the ectocervix. The remainder of the exam is normal.\nQuestion: Which of the following organisms will most likely be revealed on wet mount microscopy? Options: {A: Budding yeasts cells and/or pseudohyphae, B: Epithelial cells covered by numerous bacterial cells, C: Motile round or oval-shaped microorganisms, D: Numerous rod-shaped bacteria, E: Chains of cocci}\nAnswer:\nC'}]

In [40]:
response30 = """Answer the question using the context below.
Context: A 53-year-old woman with hypertension and hyperlipidemia comes to the physician because of generalized reddening of her skin and itching for the past 2 weeks. Her symptoms occur every evening before bedtime and last for about 30 minutes. Three months ago, atorvastatin was stopped after she experienced progressively worsening neck and back pain. Statin therapy was reinitiated at lower doses 3 weeks ago but had to be stopped again after her musculoskeletal symptoms recurred. Her menses occur irregularly at 2–3 month intervals and last for 3–4 days. She has smoked one pack of cigarettes daily for the past 30 years. Her current medications include lisinopril and niacin. Her brother died of colonic adenocarcinoma, and her father died of small cell lung cancer. She is 169 cm (5 ft 6 in) tall and weighs 83 kg (183 lb); BMI is 29 kg/m2. Her vital signs are within normal limits. Physical examination shows no abnormalities. Serum lipid studies show:
Total cholesterol 247 mg/dL
HDL-cholesterol 39 mg/dL
LDL-cholesterol 172 mg/dL
Triglycerides 152 mg/dL
Question: Which of the following is the most appropriate next step in management? Options: {A: Administer ibuprofen, B: Measure urine hydroxyindoleacetic acid levels, C: Measure urine metanephrine levels, D: Switch lisinopril to hydrochlorothiazide, E: Switch niacin to fenofibrate}
Answer:
"""
med_output31 = med_pipeline(response30)
med_output31

[{'generated_text': 'Answer the question using the context below.\nContext: A 53-year-old woman with hypertension and hyperlipidemia comes to the physician because of generalized reddening of her skin and itching for the past 2 weeks. Her symptoms occur every evening before bedtime and last for about 30 minutes. Three months ago, atorvastatin was stopped after she experienced progressively worsening neck and back pain. Statin therapy was reinitiated at lower doses 3 weeks ago but had to be stopped again after her musculoskeletal symptoms recurred. Her menses occur irregularly at 2–3 month intervals and last for 3–4 days. She has smoked one pack of cigarettes daily for the past 30 years. Her current medications include lisinopril and niacin. Her brother died of colonic adenocarcinoma, and her father died of small cell lung cancer. She is 169 cm (5 ft 6 in) tall and weighs 83 kg (183 lb); BMI is 29 kg/m2. Her vital signs are within normal limits. Physical examination shows no abnormaliti

In [70]:
actual_answers = ["E", "A", "A", "A", "E", "C", "E", "A", "E", "E", "C", "B", "B", "B", "E", "D", "B", "B", "A", "D", "A", "A", "B", "C", "D", "B", "B", "B", "B", "B", "C", "A"]

obtained_answers = ["A"]


for i in range(2, 32):
    
    var_name = f'med_output{i}'
    
    obtained_answer = globals().get(var_name, '')
    
    
    obtained_answers.append(obtained_answer[0]['generated_text'].split('\n')[-1].strip()[-1])


correct_answers = sum(1 for actual, obtained in zip(actual_answers, obtained_answers) if actual == obtained)

total_questions = len(actual_answers)

accuracy = (correct_answers / total_questions) * 100
print(obtained_answers)
print(f'Accuracy: {accuracy}%')

['A', 'A', 'C', 'A', 'E', 'C', 'A', 'A', 'D', 'A', 'A', 'I', 'C', 'C', 'B', 'C', 'A', 'A', 'C', 'D', 'A', 'A', 'A', 'C', 'D', 'A', 'B', 'A', 'B', 'C', 'A']
Accuracy: 37.5%


In [ ]:
sample_output

[{'generated_text': 'Answer the question using the context below.\nContext: Gazpacho is a cold soup and drink made of raw, blended vegetables. Most gazpacho includes stale bread, tomato, cucumbers, onion, bell peppers, garlic, olive oil, wine vinegar, water, and salt. Northern recipes often include cumin and/or pimentón (smoked sweet paprika). Traditionally, gazpacho was made by pounding the vegetables in a mortar with a pestle; this more laborious method is still sometimes used as it helps keep the gazpacho cool and avoids the foam and silky consistency of smoothie versions made in blenders or food processors.\nQuestion: What modern tool is used to make gazpacho?\nAnswer:\nThe modern tool used to make gazpacho is a food processor. The process is similar to that of making smoothies, but the ingredients are blended instead of pounded. The result is a smoother, more consistent texture.\n\nBased on the text material above, generate the response to the following quesion or instruction: Can